In [ ]:
try:
    import re
    import threading
    import pandas as pd
    import time as time
    import database as db
    from bs4 import BeautifulSoup
    from unidecode import unidecode
    from selenium import webdriver
    from selenium.webdriver.common.by import By
    from selenium.webdriver.common.keys import Keys 
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
except Exception as e:
    print(e)

In [ ]:
class BotLattes:
    def __init__(self, formacao, inicio=0, limitePaginas=None, curriculo=1, nome="Bot"):
        print(f"----- Start {nome} - {inicio} <-> {limitePaginas} -----")
        self.init = time.perf_counter()
        self.data = db.DataBase()
        self.nome = nome
        self.limite = None
        self.inicio = inicio
        self.fim = limitePaginas
        self.delimiters = ".", ",", ";", ":"
        self.regexPattern = '|'.join(map(re.escape, self.delimiters))
        self.getData(formacao, inicio=inicio, limitePaginas=limitePaginas, curriculo=curriculo)
        
    def close(self):
        try:
            self.data.saveInFile(2018)
            self.data.saveInFile(2019)
            self.data.saveInFile(2020)
            self.data.saveInFile(2021)
            self.data.connectionfile.commit()
            self.data.connection.close()
        except Exception as e:
            print(f"Bot {self.name} - Error:\n{e}")
            return
        print(f"----- End {self.nome} - {self.inicio} <-> {self.fim} ----")
        print(f"Tempo de execução: {time.perf_counter()-self.init:.2f}")
        
    def getData(self, formacao, inicio=0, limitePaginas=None, curriculo=1):
        op = Options()
        op.headless = True
        navegador = webdriver.Chrome(options=op)
        navegador.get("http://buscatextual.cnpq.br/buscatextual/busca.do")
        
        navegador.find_element_by_xpath('//*[@id="tit_simples"]/a').click()

        #Tags de pesquisa
        if formacao != "doutorado":
            navegador.find_element_by_xpath('//*[@id="buscarDemaisAvancada"]').send_keys(Keys.SPACE) #Buscar demais pesquisadores

            if formacao == "mestrado":
                navegador.find_element_by_xpath('//*[@id="filtro2"]').click()
                time.sleep(1)
                navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').click()
                navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
                navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
                navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.DOWN)
                navegador.find_element_by_xpath('//*[@id="nivelFormacao"]').send_keys(Keys.ENTER)
                time.sleep(1)
                navegador.find_element_by_xpath('//*[@id="preencheCategoriaNivelBolsa"]')

        
        navegador.find_element_by_xpath('//*[@id="buscarEstrangeirosAvancada"]').click()

        #Caixa de texto
        buscar = navegador.find_element_by_xpath('//*[@id="busca_avancada"]/div/div[2]/div/fieldset/textarea')
        buscar.send_keys("(2018 OR 2019 OR 2020 OR 2021)")
        buscar.send_keys(Keys.ENTER)

        time.sleep(2)

        passadas = []

        janelaAtual = navegador.current_window_handle
        passadas.append(navegador.window_handles)

        #Variaveis de controle da paginação
        pagina = 1
        pag = 2
        troca = True
        delay_max = 180

        WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '/html/body/form/div/div[4]/div/div/div/div[3]/div/div[1]/b[1]')))
        self.limite = int(navegador.find_element_by_xpath('/html/body/form/div/div[4]/div/div/div/div[3]/div/div[1]/b[1]').get_attribute("innerHTML").splitlines()[0])
        self.limite = int(self.limite/10 if self.limite%10==0 else (self.limite/10)+1)

        if inicio!=0:
            WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]')))
            navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
            pagina = inicio
            link = navegador.current_url.split(";")
            link[0] = link[0].replace(str(10), str((pagina-1)*10))
            link = link[0]+";"+link[1]

            pag = inicio+1
            if inicio<11:
                pag = inicio+1
            elif inicio%10==0:
                pag = 13
            else:
                div = "1"+("0"*(len(str(inicio))-1))
                pag = int((inicio+1)/int(div))+2
            navegador.get(link)
            troca = False
            inicio = 0

        while(True):
            #Troca de pagina        
            if troca:
                try:
                    if pagina!=1 and pag<=11 and pagina<=11:
                        WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]')))
                        navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                        pag = pag+1 if pag<11 else 3

                    if pagina>11 and pag>2:
                        WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]')))
                        navegador.find_element_by_xpath(f'/html/body/form/div/div[4]/div/div/div/div[3]/div/div[4]/a[{pag}]').click()
                        pag = pag+1 if pag<13 else 3
                except Exception as e:
                    print(e)
                    break

            troca = True
            endFor = False
            for i in range(curriculo,11):
                try:
                    #Selecionar curriculo
                    WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a')))
                    navegador.find_element_by_xpath(f'//html/body/form/div/div[4]/div/div/div/div[3]/div/div[3]/ol/li[{i}]/b/a').click()

                    time.sleep(1)

                    #Abrir curriculo
                    WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="idbtnabrircurriculo"]')))
                    navegador.find_element_by_xpath('//*[@id="idbtnabrircurriculo"]').click()

                except Exception as e:
                    print(e)
                    endFor = True
                    break

                time.sleep(2)

                try:
                    for janela in navegador.window_handles:
                        if janela not in passadas:
                            navegador.switch_to.window(janela)

                except:
                    navegador.switch_to.window(janelaAtual)
                    navegador.get(navegador.current_url)
                    i -= 1
                    continue

                try:
                    #Pegar dados
                    WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[3]/div/div/div/div[1]/h2'))) 
                    site = BeautifulSoup(navegador.page_source, 'html.parser')
                    artigos2018 = []
                    artigos2019 = []
                    artigos2020 = []
                    artigos2021 = []


                    for artigo in site.findAll('div', attrs={'class':['artigo-completo']}):
                        maior = max(re.split(self.regexPattern, artigo.text), key=len)
                        
                        if "2018" in artigo.text:
                            artigos2018.append(maior)
                            
                        elif "2019" in artigo.text:
                            artigos2019.append(maior)
                            
                        elif "2020" in artigo.text:
                            artigos2020.append(maior)
                            
                        elif "2021" in artigo.text:
                            artigos2021.append(maior)

                    nome = site.findAll('h2', attrs={'class':['nome']})[0].text
                except Exception as e:
                    time.sleep(1)
                    i-=1
                    print(e)
                    continue

                #Inserir no banco de dados
                if artigos2018 != []:
                    self.data.insertData(nome, formacao, artigos2018, 2018)
                if artigos2019 != []:
                    self.data.insertData(nome, formacao, artigos2019, 2019)
                if artigos2020 != []:
                    self.data.insertData(nome, formacao, artigos2020, 2020)
                if artigos2021 != []:
                    self.data.insertData(nome, formacao, artigos2021, 2021)

                navegador.close()

                time.sleep(1)

                navegador.switch_to.window(janelaAtual)

                #Fechar curriculo
                WebDriverWait(navegador, delay_max).until(EC.presence_of_element_located((By.XPATH, '//*[@id="idbtnfechar"]')))
                navegador.find_element_by_xpath('//*[@id="idbtnfechar"]').click()

            if endFor:
                break

            self.data.connection.commit()

            #Limitador de pagínas
            if pagina == limitePaginas:
                break

            pagina += 1
            curriculo = 1

        navegador.quit()
        if pagina == limitePaginas or pagina == self.limite:
            self.close()
        else:
            self.getData(formacao, inicio=pagina, curriculo=i, limitePaginas=limitePaginas)

In [ ]:
def gerarDivs(tamanho, qtTotal):
    listDivs = []
    for i in range(1, qtTotal+1, tamanho):
        listDivs.append([False, i, i+(tamanho-1)])
    return listDivs

Execução 1: 24/10 10:30 - 500 páginas (1-500)

In [ ]:
#Divisão da quantidade de páginas
#qtmax - Quantidade total de páginas que será acessada pelo bot
#tamParte - Tamanho de cada "bloco" de páginas a ser dividido nas threads

qtmax = 500
tamParte = 50
divs = gerarDivs(tamParte, qtmax)

#Variaveis de controle
lastpos = 0

bot1 = True
bot2 = True
bot3 = True
bot4 = True

th1 = threading.Thread(target=lambda:BotLattes("doutorado"))
th2 = threading.Thread(target=lambda:BotLattes("doutorado"))
th3 = threading.Thread(target=lambda:BotLattes("doutorado"))
th4 = threading.Thread(target=lambda:BotLattes("doutorado"))

#Controle das threads
while(bot1, bot2, bot3, bot4):
    if lastpos < len(divs):
        if not th1.is_alive() and bot1:
            if not divs[lastpos][0]:
                th1 = threading.Thread(target=lambda:BotLattes("doutorado", inicio=divs[lastpos][1], limitePaginas=divs[lastpos][2], nome="Bot 1"))
                th1.start()
                divs[lastpos][0] = True
                lastpos += 1
            else:
                bot1 = False
                
        if not th2.is_alive() and bot2:
            if not divs[lastpos][0]:
                th2 = threading.Thread(target=lambda:BotLattes("doutorado", inicio=divs[lastpos][1], limitePaginas=divs[lastpos][2], nome="Bot 2"))
                th2.start()
                divs[lastpos][0] = True
                lastpos += 1
            else:
                bot2 = False
                
        if not th3.is_alive() and bot3:
            if not divs[lastpos][0]:
                th3 = threading.Thread(target=lambda:BotLattes("doutorado", inicio=divs[lastpos][1], limitePaginas=divs[lastpos][2], nome="Bot 3"))
                th3.start()
                divs[lastpos][0] = True
                lastpos += 1
            else:
                bot3 = False
                
        if not th4.is_alive() and bot4:
            if not divs[lastpos][0]:
                th4 = threading.Thread(target=lambda:BotLattes("doutorado", inicio=divs[lastpos][1], limitePaginas=divs[lastpos][2], nome="Bot 4"))
                th4.start()
                divs[lastpos][0] = True
                lastpos += 1
            else:
                bot4 = False
    else:
        break

In [ ]:
data = db.DataBase()

In [ ]:
data.selectData(2018)

In [ ]:
data.selectData(2019)

In [ ]:
data.selectData(2020)

In [ ]:
data.selectData(2021)

In [ ]:
data.dellAll(2018)
data.dellAll(2019)
data.dellAll(2020)
data.dellAll(2021)